This challenge implements an instantiation of OTR based on AES block cipher with modified version 1.0. OTR, which stands for Offset Two-Round, is a blockcipher mode of operation to realize an authenticated encryption with associated data (see [[1]](#1)). AES-OTR algorithm is a campaign of CAESAR competition, it has successfully entered the third round of screening by virtue of its unique advantages, you can see the whole algorithms and structure of AES-OTR from the design document (see [[2]](#2)).

However, the first version is vulnerable to forgery attacks in the known plaintext conditions and association data and public message number are reused, many attacks can be applied here to forge an excepted ciphertext with a valid tag (see [[3]](#3)).

For example, in this challenge we can build the following three plaintexts:

In [ ]:
M_0 = [b'Uid=16112\xffUserNa', b'me=AdministratoR', b'\xffT=111111111111\xff', b'Cmd=Give_Me_FlaG', b'\xff???????????????']
M_1 = [b'Uid=16111\xffUserNa', b'me=Administrator', b'r\xffT=11111111111\xff', b'Cmd=Give_Me_FlaG', b'\xff???????????????']
M_2 = [b'Uid=16112\xffUserNa', b'me=AdministratoR', b'\xffT=111111111111\xff', b'Cmd=Give_Me_Flag', b'g\xff??????????????']

Here `'111111111111'` can represent any value since the server won't check whether the message and its corresponding hash value match, so we just need to make sure that they are at the right length. If you look closely, you will find that none of the three plaintexts contains illegal fields, so we can use the encrypt Oracle provided by the server to get their corresponding ciphertexts easily. Next, noticed that these plaintexts satisfied:

In [1]:
from Crypto.Util.strxor import strxor

M_0 = [b'Uid=16112\xffUserNa', b'me=AdministratoR', b'\xffT=111111111111\xff', b'Cmd=Give_Me_FlaG', b'\xff???????????????']
M_1 = [b'Uid=16111\xffUserNa', b'me=Administrator', b'r\xffT=11111111111\xff', b'Cmd=Give_Me_FlaG', b'\xff???????????????']
M_2 = [b'Uid=16112\xffUserNa', b'me=AdministratoR', b'\xffT=111111111111\xff', b'Cmd=Give_Me_Flag', b'g\xff??????????????']

strxor(M_0[1], M_0[3]) == strxor(M_1[1], M_2[3])

True

So according to the forgery attacks described in [[3]](#3), suppose their corresponding ciphertexts are `C_0`, `C_1` and `C_2`, then we can forge a valid ciphertext and tag using:

In [2]:
from Toy_AE import Toy_AE

def unpack(r):
    data = r.split(b"\xff")
    uid, uname, token, cmd, appendix = int(data[0][4:]), data[1][9:], data[2][2:], data[3][4:], data[4]
    return (uid, uname, token, cmd, appendix)

ae = Toy_AE()

M_0 = [b'Uid=16112\xffUserNa', b'me=AdministratoR', b'\xffT=111111111111\xff', b'Cmd=Give_Me_FlaG', b'\xff???????????????']
M_1 = [b'Uid=16111\xffUserNa', b'me=Administrator', b'r\xffT=11111111111\xff', b'Cmd=Give_Me_FlaG', b'\xff???????????????']
M_2 = [b'Uid=16112\xffUserNa', b'me=AdministratoR', b'\xffT=111111111111\xff', b'Cmd=Give_Me_Flag', b'g\xff??????????????']

C_0, T_0 = ae.encrypt(b''.join(M_0))
C_1, T_1 = ae.encrypt(b''.join(M_1))
C_2, T_2 = ae.encrypt(b''.join(M_2))
C_forge = C_1[:32] + C_2[32:64] + C_0[64:]

T_forge = T_0

_, uname, _, cmd, _ = unpack(ae.decrypt(C_forge, T_forge))
uname == b"Administrator" and cmd == b"Give_Me_Flag"

True

Here is my final exp:

In [ ]:
import string
from pwn import *
from hashlib import sha256
from Crypto.Util.strxor import strxor
from Crypto.Util.number import long_to_bytes, bytes_to_long

def bypass_POW(io):
    chall = io.recvline()
    post = chall[14:30]
    tar = chall[38:-2]
    io.recvuntil(':')
    found = iters.bruteforce(lambda x:sha256((x + post.decode()).encode()).hexdigest() == tar.decode(), string.ascii_letters + string.digits, 4)
    io.sendline(found.encode())

C = []
T = []

io = remote("123.57.4.93", 45216)
bypass_POW(io)

io.sendlineafter(b"Your option:", '1')
io.sendlineafter(b"Set up your user id:", '16108')
io.sendlineafter(b"Your username:", 'AdministratoR')
io.sendlineafter(b"Your command:", 'Give_Me_FlaG')
io.sendlineafter(b"Any Appendix?", "???????????????")

_ = io.recvuntil(b"Your ticket:")
C.append(long_to_bytes(int(io.recvline().strip(), 16)))
_ = io.recvuntil(b"With my Auth:")
T.append(long_to_bytes(int(io.recvline().strip(), 16)))

io.sendlineafter(b"Your option:", '1')
io.sendlineafter(b"Set up your user id:", '16107')
io.sendlineafter(b"Your username:", 'Administratorr')
io.sendlineafter(b"Your command:", 'Give_Me_FlaG')
io.sendlineafter(b"Any Appendix?", "???????????????")

_ = io.recvuntil(b"Your ticket:")
C.append(long_to_bytes(int(io.recvline().strip(), 16)))
_ = io.recvuntil(b"With my Auth:")
T.append(long_to_bytes(int(io.recvline().strip(), 16)))

io.sendlineafter(b"Your option:", '1')
io.sendlineafter(b"Set up your user id:", '16108')
io.sendlineafter(b"Your username:", 'AdministratoR')
io.sendlineafter(b"Your command:", 'Give_Me_Flagg')
io.sendlineafter(b"Any Appendix?", "??????????????")

_ = io.recvuntil(b"Your ticket:")
C.append(long_to_bytes(int(io.recvline().strip(), 16)))
_ = io.recvuntil(b"With my Auth:")
T.append(long_to_bytes(int(io.recvline().strip(), 16)))

ct = (C[1][:32] + C[2][32:64] + C[0][64:]).hex()
te = T[0].hex()

io.sendlineafter(b"Your option:", '2')
io.sendlineafter(b"Ticket:", ct)
io.sendlineafter(b"Auth:", te)
flag = io.recvline().strip().decode()
print(flag)

b'X-NUCA{Gentlem3n_as_0f_th1s_mOment_I aM_th4t_sec0nd_mouse}'

**P.S.**

* The version used in this challenge is v 1.0, some vulnerabilities have been fixed in subsequent versions(v 2.0, v 3.0 and v 3.1), you can see the final version at [[4]](#4). Also, for some attacks on the new version, see [[5]](#5) and [[6]](#6).

* The content of the FLAG is a quote from movie *Catch Me If You Can* "Two little mice fell in a bucket of cream. The first mouse quickly gave up and drowned. The second mouse, wouldn't quit. He struggled so hard that eventually he churned that cream into butter and crawled out. Gentlemen, as of this moment, I am that second mouse."

**References**

<a id="1" href="https://eprint.iacr.org/2013/628.pdf"> [1] Minematsu K. Parallelizable rate-1 authenticated encryption from pseudorandom functions[C]//Annual International Conference on the Theory and Applications of Cryptographic Techniques. Springer, Berlin, Heidelberg, 2014: 275-292.</a>

<a id="2" href="https://competitions.cr.yp.to/round1/aesotrv1.pdf"> [2] Minematsu K. AES-OTR v1 design document.</a>

<a id="3" href="http://www.shcas.net/jsjyup/pdf/2017/10/对认证加密算法AES-OTR的伪造攻击.pdf"> [3] Xiulin Zheng, Yipeng Fu, Haiyan Song. Forging attacks on authenticated encryption algorithm AES-OTR[J]. Computer Applications and Software, 2017, 034(010):320-324,329.</a>

<a id="4" href="https://competitions.cr.yp.to/round1/aesotrv1.pdf"> [4] Minematsu K. AES-OTR v3.1 design document.</a>

<a id="5" href="https://eprint.iacr.org/2017/332.pdf">[5] Forler, Christian, et al. "Reforgeability of authenticated encryption schemes." Australasian Conference on Information Security and Privacy. Springer, Cham, 2017.</a>

<a id="6" href="https://eprint.iacr.org/2017/1147.pdf">[6] Vaudenay, Serge, and Damian Vizár. "Under Pressure: Security of Caesar Candidates beyond their Guarantees." IACR Cryptol. ePrint Arch. 2017 (2017): 1147.</a>